In [5]:
import warnings
import os
import snowflake.connector

warnings.filterwarnings("ignore", category=DeprecationWarning)

# Replace these with your Snowflake account credentials and connection details
account = 'helpathome.east-us-2.azure'  # Replace with your Snowflake account URL
warehouse = 'DEMO_WH'
database = 'DISC_PROD'
schema = 'schemas'
username = 'dsingh'  # Replace with your Snowflake username
password = 'Hah@2023'  # Replace with your Snowflake password

# List of schemas to iterate through
schemas = ['1UP_BISON','1UP_HEALTH','ACTIVE_DIRECTORY','ADAPTIVENURSINGEXPORT','ADI','ADMINPAYROLL','ADP','ALAYACARE','AMERIHEALTH','APPBASEDATA','APTITIVE_DEV','ASR','AXXESS','BI_REPOSITORY','CCSI','CENTENE',
'CLEARCARE','COSTALSYNCDATA','DATAFLEXSYNCDATA','DATA_CLEANSING','DATA_VAULT','DEVERO','DOCEBO','EMPEONEDISON','EMPEONPREFERRED','EMPLOYEESETUP','FILEPRO','FINANCE',
'GENERATIONSALLIANCE','GPSYNCDATA','HAHUSERS','HAH_APP','HAH_REPORTING','HEALTH_NAVIGATOR','HEALTH_STREAM','HHAEXCHANGEEDISON','HHAEXCHANGEOPENSYSTEMS','HHAEXCHANGEOSHAH',
'HHAEXCHANGEPREFERRED','HSX','ICD','JAZZHR','LANDING_8X8','MATRIXCARE','MATRIXCARE_FULLLOAD','MOLINA','NAVISION','OKTA','OLYMPUS','PARADOX','PAYLOCITY',
'PAYOR_CONTRACT_UI','PUBLIC','QUALTRICS','QUALTRICS_SURVEYS','RISKONNECT','SANDATAEXCHANGE','SANDATAIMPORT','STAGE','TRUSTPOINTDATA','TRUSTPOINTDATA_HIST',
'USZIPS','UTIL','VIVENTIUM','WORKDAY','ZENDESK']

#  Create the SQL_Files and Table folders
sql_files_dir = "SQL_Files"
table_dir = os.path.join(sql_files_dir, "Table")

if not os.path.exists(sql_files_dir):
    os.mkdir(sql_files_dir)

if not os.path.exists(table_dir):
    os.mkdir(table_dir)

# Establish a connection to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database
)

for schema in schemas:
    # Create a cursor to execute SQL queries
    cursor = conn.cursor()

    # Query Snowflake to get a list of tables in the specified database and schema
    table_query = f'''
        SELECT (table_catalog || '.' || table_schema || '.' || table_name) table_name
        FROM information_schema.tables
        WHERE table_schema = '{schema}' AND table_catalog = '{database}' AND TABLE_TYPE = 'BASE TABLE'
    '''

    # Execute the query to get the list of tables
    cursor.execute(table_query)

    # Fetch the results
    tables = cursor.fetchall()

    # Close the cursor when done with the table query
    cursor.close()

    if tables:
        # Iterate through the tables and retrieve DDL statements
        for table_info in tables:
            table_name = table_info[0]

            # Construct the fully qualified table name
            fully_qualified_table_name = f'{table_name}'
            try:
            
                # Query to retrieve the DDL statement for the table
                ddl_query = f'''
                SELECT GET_DDL('TABLE', '{table_name}',true)
                '''

                # Create a new cursor for the DDL query
                cursor = conn.cursor()

                # Execute the DDL query
                cursor.execute(ddl_query)

                # Fetch the DDL statement
                ddl_statement = cursor.fetchone()[0]

                # Modify the DDL statement to include the database and schema names
                modified_ddl_statement = ddl_statement.replace(f'create or replace TABLE {table_name}',
                                                            f'CREATE OR REPLACE TABLE {fully_qualified_table_name}')

                # Step 2: Create a .sql file and write the DDL statement to it
                sql_file_name = f"{table_dir}/{fully_qualified_table_name}.sql"
                with open(sql_file_name, 'w') as sql_file:
                    sql_file.write(modified_ddl_statement)

                # Step 3: Print the file name (database.schema.table name)
                print(f"Generated SQL file: {sql_file_name}")

                # Close the cursor for the DDL query
                cursor.close()
            except snowflake.connector.errors.ProgrammingError as e:
                print('Table does not exist or not authorized ', fully_qualified_table_name)
    else:
        print(f"No tables found in the specified schema '{schema}' in database '{database}'.")

# Close the connection
conn.close()

Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_ALLERGYINTOLERANCE.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_COVERAGE.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_ENCOUNTER.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_OBSERVATION.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_ORGANIZATION.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_PATIENT.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_PRACTITIONER.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_PROCEDURE.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.FHIR_R4_SERVICEREQUEST.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.HIST_FHIR_R4_CONDITION.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.HIST_FHIR_R4_DIAGNOSTICREPORT.sql
Generated SQL file: SQL_Files\Table/DISC_PROD.1UP_BISON.HIST_FHIR_R4_ENCOUNTER.sql
Generated SQL file: SQL